In [1]:
import pandas as pd
import re
from itertools import chain

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
features = pd.read_excel('data//features.xlsx', index_col=0).T
features
for c in features.columns:
    print(c)

letter,p,t,tʃ,k,b,d,dʒ,g,f,v,...,ɑ,ɒ,ɪ,ʏ,ᵻ,ʊ,ε,ʌ,ɔ,a
syl,-,-,-,-,-,-,-,-,-,-,...,+,+,+,+,+,+,+,+,+,+
son,-,-,-,-,-,-,-,-,-,-,...,+,+,+,+,+,+,+,+,+,+
cons,+,+,+,+,+,+,+,+,+,+,...,-,-,-,-,-,-,-,-,-,-
cont,-,-,-,-,-,-,-,-,+,+,...,+,+,+,+,+,+,+,+,+,+
delayed release,-,-,+,-,-,-,+,+,-,-,...,-,-,-,-,-,-,-,-,-,-
lat,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
nas,-,-,-,-,-,-,-,-,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
voi,-,-,-,-,+,+,+,+,-,+,...,+,+,+,+,+,+,+,+,+,+
constricted glottis,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
spread glottis,-,-,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


p
t
tʃ
k
b
d
dʒ
g
f
v
θ
ð
s
z
ʃ
ʒ
h
ʔ 
m
n
ɲ
ŋ
r
l
j
w
ʔ
i
y
ɨ
u
e
ø
ə
o
æ
œ
ɑ
ɒ
ɪ 
ʏ
ᵻ
ʊ
ε
ʌ
ɔ
a


In [3]:
features[['b', 'd','g']]

letter,b,d,g
syl,-,-,-
son,-,-,-
cons,+,+,+
cont,-,-,-
delayed release,-,-,+
lat,-,-,-
nas,-,-,-
voi,+,+,+
constricted glottis,-,-,-
spread glottis,-,-,-


In [4]:
def build_letter_list(features):
    
    group = []
    letter = None
    
    while letter != '0':
        letter = input('Enter a letter(0 to exit): ')
        if letter == '0':
            break
        elif letter not in features.columns:
            print('Not in data.')
            continue
        else:
            group.append(letter.strip().lower())
    
    return group

In [22]:
# Compares groups of phones and gives similar features within groups but different across groups
# to just get commonalities of a group, enter '0' as the first letter in Group2
def get_features(features):    
    
    print('Group 1:')    
    group1 = build_letter_list(features)
    
    print('\nGroup2:')
    group2 = build_letter_list(features)
    
    group1_list = [(a, list(set(b))[0]) for a, b in features[group1].iterrows() if len(set(b)) == 1]
    feature, value = zip(*group1_list)
    group1_similar = pd.Series(value, feature, name=', '.join(group1))
    
    if len(group2) > 0:        
        group2_list = [(a, list(set(b))[0]) for a, b in features[group2].iterrows() if len(set(b)) == 1]
        feature, value = zip(*group2_list)
        group2_similar = pd.Series(value, feature, name=', '.join(group2))
    
        different_features = []

        for a, b in pd.concat([group1_similar, group2_similar], axis=1).iterrows():
            if a in group1_similar.index and a in group2_similar.index:
                if b[0] != b[1] and isinstance(b[0], str) and isinstance(b[1], str):
                    different_features.append([a, b[0], b[1]])

        compare = pd.DataFrame(different_features, 
                               columns=['features', group1_similar.name, group2_similar.name]).set_index('features')
        
        
        return compare
            
    else:
        group1_similar = pd.DataFrame(group1_similar)
        return group1_similar

    
a = get_features(features)
a

Group 1:
Enter a letter(0 to exit): p
Enter a letter(0 to exit): t
Enter a letter(0 to exit): k
Enter a letter(0 to exit): 0

Group2:
Enter a letter(0 to exit): b
Enter a letter(0 to exit): d
Enter a letter(0 to exit): g
Enter a letter(0 to exit): 0


,"p, t, k","b, d, g"
features,,
voi,-,+
